**1. Combine a rule-based and a machine-learning chatbot (like those of the example code), using rules and training data of your choice (you may retain the example setup if you wish) to implement a bot that answers clear-cut situations based on rules and defaults to the responses from a fitted neural network when no rule matches. Interact with it and discuss the strengths and the weaknesses of your bot. Please include code snippets and an example dialogue in your response.**

https://www.techwithtim.net/tutorials/ai-chatbot/part-1/

In [ ]:
!pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 107 kB 27.7 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=497bbe19048bf58dc1aac7d9bccb9573f0d37e11ee23ee869615703571bf53bb
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [ ]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import tensorflow as tf
import numpy
import tflearn
import tensorflow
import random

# Load the json file

import json
with open('intents.json') as file:
    data = json.load(file)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
# Training data with the json file

data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'hours',
   'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day']},
  {'tag': 'payments',
   'patterns': ['Do you take credit cards?',
    'Do you accept Mastercard?',
    'Are you cash only?'],
   'responses': ['We accept VISA, Mastercard and AMEX',
    'We accept most major credit c

In [ ]:
# Extract the data

words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [ ]:
# Word Stemming to reduce the vocabulary of our model and attempt to find the more general meaning behind sentences.

words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

In [ ]:
# Bag of words
# We need some way to represent our sentences with numbers and this is where a bag of words comes in.

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [ ]:
# Convert our training data and output to numpy arrays

training = numpy.array(training)
output = numpy.array(output)

In [ ]:
# Develop a model
# The goal of our network will be to look at a bag of words and give a class that they belong too (one of our tags from the JSON file).

tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
# Training & saving the model

model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 2999  | total loss: 0.00234 | time: 0.004s
| Adam | epoch: 1000 | loss: 0.00234 - acc: 1.0000 -- iter: 16/20
Training Step: 3000  | total loss: 0.00236 | time: 0.006s
| Adam | epoch: 1000 | loss: 0.00236 - acc: 1.0000 -- iter: 20/20
--


In [ ]:
# Make predictions:
# – Get some input from the user
# – Convert it to a bag of words
# – Get a prediction from the model
# – Find the most probable class
# – Pick a response from that class

def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

In [ ]:
# run the chatbot
chat()

Start talking with the bot (type quit to stop)!
You: Hello
Good to see you again
You: Are you open Sunday?
Our hours are 9am-9pm every day
You: Can I pay with credit card?
We accept VISA, Mastercard and AMEX
You: Do you have chicken to sell?
We accept most major credit cards
You: ok thanks!
My pleasure
You: ciao
We accept VISA, Mastercard and AMEX
You: Bye
Bye! Come back again soon.
You: quit


J’ai appliqué un chatbot très similaire au cours selon le tutoriel ci-dessus. Importation d’un fichier json pour définir les questions / réponses du chatbot. Extraction du fichier. Les mots sont stemming pour réduire le vocabulaire pour mon modèle. Utilisation de bag of word pour représenter mes phrases en nombre dans mon modèle. Développement d’un modèle à l’aide de tensorflow. Pour faire les prédictions, il va prendre les phrases de l’utilisateur, les convertir en bag of words pour faire une prédiction selon mon modèle. Il trouvera la réponse la plus probable en envoyant la classe avec la probabilité la plus grande en guise de réponse.

Les réponses selon ma structure définie dans mon fichier json fonctionne bien. Même si une phrase n’est pas identique à ma structure comme par exemple ‘Are you open Sunday?’, le modèle cherchera la meilleure réponse à donner qui est cohérent ‘Our hours are 9am-9pm every day’. Les phrases qui ne sont pas dans ma structure donne des réponses incohérente ‘Do you have chiken to sell? We accept most major credit cards’ ou utilisation d’une autre langue ‘ciao: We accept VISA, MasterCard and AMEX’.

**2. Does it sound like a good idea to continue training a chatbot based on the reactions that its responses elicit from the users in a live environment? What would you gain by doing this? What could go wrong? In what context could it make sense to do this?**

https://phys.org/news/2021-07-customers-react-chatbots.html

Selon cette référence, si les problèmes questionnés par l’utilisateur sont importants ou critiques, la réaction est négative du client lorsqu’il apprend que c’est un chatbot. Ceci affaiblie la confiance du client. Par contre, si le problème n’est pas résolu dans les questions, l’utilisateur a une réaction positive lorsqu’il sait que c’est un chatbot. L’utilisateur comprend plus facilement la cause de l’erreur et il plus indulgent è l’erreur d’une technologie qu’un humain.

https://marutitech.com/benefits-chatbot/

Les bénéfices d’un chatbot pour une compagnie:

•	Réduction des coûts : Permet aux entreprises de gérer une quantité importante de demandes clients dans des délais courts. Investissement d’un chatbot inférieur au modèle traditionnel de service à la clientèle.

•	Offrir aux clients du site web une assistance contextuelle à valeur ajoutées : Expérience personnalisée aux clients.

•	Une meilleure analyse des données clients : L’entreprise peut analyser les performances du bot en résultats commerciaux / ventes générées, informations détaillées sur la façon dont les gens s’engagent avec l’entreprise et leur demande. 

•	Amélioration de l’engagement des clients et des ventes : La structure flexible des chatbots les rend faciles à les intégrer à d’autres systèmes, ce qui augmente l’engagement. Expérience rapide et sans tracas laisse le client heureux et satisfait.

•	Gagner du temps : En résolvant les questions de base. Seules les requêtes complexes nécessitant une intervention humaine sont dirigées vers une équipe d’assistance.

Application dans de multiple cas et industries :

•	Marketing, vente, service à la clientèle, service d’assistance IT.

•	Service de la santé, banque et finance, éducation, ressource humaine, commerce de détail, tourisme.

Il faut cependant faire attention aux problématiques d’étiques comme par exemple la sécurité des utilisateurs, biais textuels, vie privée des gens, générer des stéréotypes ou être nuisibles. Voir dans la question 3 pour plus de développement sur ces points.

En somme, le chatbot me semble favorable.


**3. How about training a chatbot in a completely non-supervised manner (meaning that no labeled data is ever used)? Does this sound feasible? What could cause difficulties with such an approach?**

https://www.searchunify.com/blog/the-rise-of-the-unsupervised-learning-based-chatbot-models/


Les 2 premières générations de chatbots sont basées sur des modèles d’apprentissages automatique supervisés.

•	Réponses dénuées de sens pour des questions non définis.

•	Étiquetage intensif des données pour la formation du modèle.


Les modèles non supervisés reconnaissent des modèles et extraient l’intention des données recherchées par eux-mêmes. Ils offrent des suggestions d’intention aux administrateurs en données. Informations en temps réel qui fournissent des données sur des paramètres tels que l’interaction des utilisateurs, les intentions, etc. Ces chatbots s’auto-apprennent et s’améliore au fur et à mesure qu’ils reçoivent de nouvelles données. Réduit le temps de formation, l’administration et la maintenance nécessaire tout en améliorant la qualité des interactions avec le client.


https://chatbotsmagazine.com/unsupervised-deep-learning-for-vertical-conversational-chatbots-c66f21b1e0f
https://arxiv.org/pdf/1506.05869v3.pdf


Une approche simple consiste à utiliser un cadre d’apprentissage profond non supervisé de type réseau neuronal récurrent séquence à séquence (seq2seq RNN). C’est un type de système (Vinyals-Le) questions/réponses issu de la traduction automatique. Le contexte est limité à la question en cours. Dans un contexte conversationnel étendu basé sur le dialogue, le contexte doit être maintenu à travers de multiples séquences de questions/réponses (exigence clé pour les systèmes de dialogue).

•	Chatbots verticaux : domaine fermé axés sur des applications verticales particulières (plus simple). 

•	Chatbots horizontaux : domaine ouvert comme Siri, Google assistant ou Alexa.


Les chatbots verticaux sont des systèmes axés sur les objectifs, ou le but de la conversation est d’obtenir des informations pour exécuter une action. La classification des intentions est moins problématique puisque le domaine vertical fournit un contexte pour la conversation. 


L’apprentissage non supervisé implique que le modèle peut être entraîné directement à partir des données historiques des transcriptions, sans qu’il soit nécessaire de recourir à un étiquetage humain. Si l’objectif est de construire un système de Q/A, nous pouvons traiter les conversations comme des paires de Q/A d’entrée, ou chaque phrase de la conversation est à la fois une réponse à la phrase précédente et une question à la phrase suivante. La taille et le contenu du vocabulaire sont un paramètre clé du modèle. Plus le vocabulaire est restreint, moins le temps de formation est long. Un petit nombre de vocabulaire entraîne un plus grand nombre d’entité unknow (données importantes pour la conversation mais qui ne peuvent pas figurer dans le vocabulaire). La gestion de ces inconnus est l’un des défis de la construction d’un chatbot vertical avec un vocabulaire spécifique. Les autres paramètres importants sont le nombre de layers, le type et le nombre de cellules utilisé pour chaque layers et le nombre autorisé pour chaque phrase.

L’avantage de ce modèle est la simplicité avec peu de besoin de règles dépendantes du domaine. Les inconvénients sont l’adaptation à la tâche de construction des chatbots, la difficulté à maintenir le contexte dans les longues conversations et à tenir un dialogue cohérent. 


https://web.stanford.edu/~jurafsky/slp3/24.pdf


L’éthique dans la conception de chatbot :

•	La sécurité des utilisateurs : ex : conseils médicaux, situation d’urgence, etc. des conseils erronés peuvent être dangereux.

•	Un système peut attaquer verbalement et générer des stéréotypes abusifs ou nuisibles. Exemple Tay bot (https://en.wikipedia.org/wiki/Tay_(bot))

•	La vie privée : L’omniprésence des chatbots à domicile signifie qu’ils peuvent souvent entendre des informations privées. Besoin d’anonymiser les informations d’identification personnelle.

•	Soulèvement d’importantes question d’égalité des sexes et biais textuels.


**4.	Find some chatbots (at least three) in the customer service options of the websites or social media of some companies or organizations that interest you. Test them out for a few minutes and write a brief review of each, speculating on how you imagine them to be implemented and if you can think of ways to improve them.**

https://www.adecco.fr/candidats/aloha/

Aloha pour la recherche d’emploi : Facilite la recherche d’emploi et un bon moyen pour Adecco de sourcer plus facilement des nouveaux profils. Sans doute un chatbot ruled-based.

Test: Elle ne comprend pas la recherche intelligence artificielle, science de données. Cependant, elle me propose 8 offres pour analyste de données. Je peux postuler ou accéder directement à l’annonce mais ils sont tous en France et certain ne me semble pas associé à ma recherche exactement (fonctions administratives, gestion comptabilité.) En tapant : gestion des données, Montréal. 5 nouvelles offres. J’accède à l’une d’entre elle (assistant contrôleur de gestion!), désolé cette offre d’emploi n’existe plus. Les deux autres essaient même chose. Je tente Montréal, cette fois un menu m’apparait avec affiner/modifier. Menu interactif : secteur / type de contrat / mots clés / Localissation. Menu déroulant avec faible éventail d’emploi (possiblement que la compagnie adecco se spécialise dans ce type d’emploi). Je tente conseiller. Suggestion d’offre d’emploi dans le paramédical en général. Désolé, cette offre d’emploi n’existe plus… Offres similaire (Villefranche De Lauragais, Haute-Garonne). Je ne sais pas si c’est seulement pour la France sachant que Adecco est établie à Montréal également.

Bref, je trouve que le bot est un bel effort, il offre des fonctionnalités de recherches qui me semble pertinentes. Cependant je trouve qu’il manque de précision dans ce qu’ils offrent versus mes mots clés tapé ou sélectionné selon mon menu. Pour tous mes tests pour voir l’affiche du poste : désolé, cette offre n’existe plus. Au bout de 5 minutes je me suis tanné et je pense que je serais plus productive à faire ma recherche manuellement sur leur site.

https://www.eviebot.com/en/

Evie bot : Elle apprend des gens (non-supervisé). Avatar féminin qui réagit à mes questions en changeant son expression faciale. Déjà un avertissement au début : n’est pas fait pour les enfants. Elle apprend et imite. Contenu social qui vise à passer le test de Turing. Peut-être grossier ou inapproprié.

Test : Je peux soit écrire ou dicter une phrase. Plusieurs langues, je choisie le français. Elle me répond en anglais. Si je lui dis que je parle en français elle change sa langue ce qui est bien. Quelque discussion avec le bot est je reçois des choses bizarres comme : tu fais quoi chaton, ta gueule, je suis menteuse, des incohérences dans les réponses, langages populaires, certaines phrases avec beaucoup de fautes d’orographes ou de prononciation, si je tape des fautes elle peut changer de langage soudainement. Parfois la connexion se perd. Ses yeux sont expressifs, son visage bouge et tente de simuler des expressions faciales.

https://blog.cfte.education/conversational-ai-examples-how-siri-alexa-google-assistant-have-human-like-conversations/

Siri : Sans doute l’un des plus sophistiqué avec Alexa et Google Assistant. Configuré pour ressembler davantage à des humains, générant des réponses plus naturelles et alignées sur des conversations humaines réelles. (Non-supervisé) Design :

•	Automatic Speech Recognition (ASR)

•	Natural Language Understanding (NLU)

•	Machine Learning to configure a response

•	Natural Language Generation (NLG)

•	Text-to-speech Software

Test: J’aimerais manger chinois: me donne des restaurants chinois à proximité de chez moi. Je demande une adresse, me donne la durée du trajet avec un itinéraire. Je demande l’anniversaire de mon père, il cherche dans mes contacts pour me donner la date et m’indiqué dans combien de jour. J’aimerais écouter de la nouvelle musique. Ouvre l’application musique sans me fournir de nouveauté. Que se passe-t-il avec Donald Trump. Me donne les dernières nouvelles sur ce personnage. Ou habite ma mère? Siri me demande le nom de ma mère et ensuite me fournit son adresse. Si je demande une opinion si elle aime la Chine, elle regrette et ne peut me répondre. Bref, de loin le meilleur chatbot testé! 
